In [26]:
import subprocess
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import requests

#pip install GitPython
import git

pd.set_option('display.max_rows',500) 

Git pull to update john hopkins dataset from github

In [27]:
g = git.cmd.Git(os.path.dirname('../data/raw/COVID-19/'))
out=g.pull()
print(out)

Already up to date.


In [28]:
# git_pull = subprocess.Popen( "/usr/bin/git pull" , 
#                      cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
#                      shell = True, 
#                      stdout = subprocess.PIPE, 
#                      stderr = subprocess.PIPE )
# (out, error) = git_pull.communicate()


# print("Error : " + str(error)) 
# print("out : " + str(out))



In [29]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)


Its shows the number of cases confirmed on each day for provices/states(give longitude and latitude) of the countries 

WebScrapping

https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html 
The website shows the number of cases in each states of the germany. 
The values are obtained through webscrapping.

Inconsistent and unreliable,since webscrapping depends on the structure of web page. A minor update in the webpage can cause error.

In [31]:
page=requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [32]:
soup=BeautifulSoup(page.content,'html.parser')

In [33]:
html_table=soup.find('table')

In [34]:
all_rows=html_table.find_all('tr')

In [35]:
final_data_list=[]

In [36]:
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)

In [37]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                                      1:'cases',
                                                                      2:'changes',
                                                                      3:'cases_per_100k',
                                                                      4:'fatal',
                                                                      5:'comment'})

In [38]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,41.574,+326,1.553,"14,0",1.862
3,Bayern,56.972,+333,1.986,"15,2",2.638
4,Berlin,11.125,+114,421,"11,2",226
5,Branden­burg,3.867,+10,70,"2,8",169
6,Bremen,1.980,+22,88,"12,9",56


API calls

https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/ef4b445a53c1406892257fe63129a8ea_0/geoservice?geometry=-33.730%2C46.270%2C49.986%2C55.886

https://npgeo-corona-npgeo-de.hub.arcgis.com/datasets/917fc37a709542548cc3be077a786c17_0/geoservice


https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json


In [39]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')
# data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')


In [40]:
json_object=json.loads(data.content)

In [41]:
type(json_object)

dict

In [42]:
json_object['features'][0]

{'attributes': {'OBJECTID_1': 1,
  'LAN_ew_AGS': '01',
  'LAN_ew_GEN': 'Schleswig-Holstein',
  'LAN_ew_BEZ': 'Land',
  'LAN_ew_EWZ': 2896712,
  'OBJECTID': 15,
  'Fallzahl': 4022,
  'Aktualisierung': 1598738400000,
  'AGS_TXT': '01',
  'GlobalID': 'fc5ba936-c95c-432c-8a33-9eb2f30b660f',
  'faelle_100000_EW': 138.847079033055,
  'Shape__Area': 45737314116.8701,
  'Shape__Length': 2881496.1295035,
  'Death': 160},
 'geometry': {'rings': [[[9.72694855217059, 53.5586543755165],
    [9.72953698118645, 53.5582212414827],
    [9.73031519620866, 53.5580193080416],
    [9.73010405438765, 53.5575809108903],
    [9.71926740492579, 53.5583267696142],
    [9.71627908277636, 53.5585171416657],
    [9.71743713604342, 53.5586307514866],
    [9.7185850167882, 53.5587521461929],
    [9.71954430073065, 53.5587011502349],
    [9.72069675929011, 53.5585657713295],
    [9.72172008862075, 53.558572225221],
    [9.72321985922525, 53.5587028673542],
    [9.72521088791592, 53.5587692928162],
    [9.726948552170

In [43]:
json_object['features'][:5]

[{'attributes': {'OBJECTID_1': 1,
   'LAN_ew_AGS': '01',
   'LAN_ew_GEN': 'Schleswig-Holstein',
   'LAN_ew_BEZ': 'Land',
   'LAN_ew_EWZ': 2896712,
   'OBJECTID': 15,
   'Fallzahl': 4022,
   'Aktualisierung': 1598738400000,
   'AGS_TXT': '01',
   'GlobalID': 'fc5ba936-c95c-432c-8a33-9eb2f30b660f',
   'faelle_100000_EW': 138.847079033055,
   'Shape__Area': 45737314116.8701,
   'Shape__Length': 2881496.1295035,
   'Death': 160},
  'geometry': {'rings': [[[9.72694855217059, 53.5586543755165],
     [9.72953698118645, 53.5582212414827],
     [9.73031519620866, 53.5580193080416],
     [9.73010405438765, 53.5575809108903],
     [9.71926740492579, 53.5583267696142],
     [9.71627908277636, 53.5585171416657],
     [9.71743713604342, 53.5586307514866],
     [9.7185850167882, 53.5587521461929],
     [9.71954430073065, 53.5587011502349],
     [9.72069675929011, 53.5585657713295],
     [9.72172008862075, 53.558572225221],
     [9.72321985922525, 53.5587028673542],
     [9.72521088791592, 53.55876929

In [44]:
full_list=[]
for pos,each_dict in enumerate(json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [45]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4022,1598738400000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,138.847079,4.573731e+10,2.881496e+06,160
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6227,1598738400000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,338.207203,2.089396e+09,4.188002e+05,266
2,3,03,Niedersachsen,Land,7982448,9,16690,1598738400000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,209.083730,1.299836e+11,4.008988e+06,661
3,4,04,Bremen,Freie Hansestadt,682986,5,1992,1598738400000,04,4132268b-54de-4327-ac1e-760e915112f1,291.660444,1.119157e+09,3.357177e+05,56
4,5,05,Nordrhein-Westfalen,Land,17932651,10,58584,1598738400000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,326.689010,8.782936e+10,2.648673e+06,1809


In [46]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

API Access via Rest service 
USA

In [47]:
url_endpoint='https://api.smartable.ai/coronavirus/stats/NP'
headers={
    'Cache-Control':'no-cache',
    'Subscription-Key':'631ee115ef754ed1a9668c1d33cd4168',
}
response=requests.get(url_endpoint,headers=headers)

In [48]:
print(response)

<Response [200]>


In [49]:
NP_dict=json.loads(response.content)
with open('../data/raw/SMARTABLE/NP_data.txt','w') as outfile:
    json.dump(NP_dict,outfile,indent=2)

In [50]:
NP_dict

{'location': {'long': 84.25,
  'countryOrRegion': 'Nepal',
  'provinceOrState': None,
  'county': None,
  'isoCode': 'NP',
  'lat': 28.1667},
 'updatedDateTime': '2020-08-30T06:00:35.2077141Z',
 'stats': {'totalConfirmedCases': 37340,
  'newlyConfirmedCases': 884,
  'totalDeaths': 207,
  'newDeaths': 12,
  'totalRecoveredCases': 20555,
  'newlyRecoveredCases': 313,
  'history': [{'date': '2020-01-22T00:00:00',
    'confirmed': 0,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-23T00:00:00',
    'confirmed': 0,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-24T00:00:00',
    'confirmed': 0,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-25T00:00:00',
    'confirmed': 1,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-26T00:00:00',
    'confirmed': 1,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-27T00:00:00',
    'confirmed': 1,
    'deaths': 0,
    'recovered': 0},
   {'date': '2020-01-28T00:00:00',
    'confirmed': 1,
    'dea